In [81]:
from langchain.prompts import PromptTemplate


match_prompt_auto_fewshot_1 = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Could you please tell me if the following two products are the same one?
    Product 1:
    Title: Kingston ValueRAM 4GB - PC3-12800 - DIMM
Description: ValueRam
    Product 2:
    Title: TARJETA DE RED PCI EXPRESS X1 INALAMBRICA TP-LINK WIRELESS 802.11N/G/B150MBPS ANTENA DESMONTA 2DBI
Description: nan
A: Let's think step by step! Product 1 is a Kingston ValueRAM 4GB memory module with a PC3-12800 specification and DIMM form factor. It is a computer component used to increase the system's memory capacity. On the other hand, Product 2 is a TP-Link wireless network card with a PCI Express x1 interface and 802.11n/g/b wireless standards. It is used to connect a computer to a wireless network. Therefore, these two products are not the same, and they serve different purposes.
Q: Could you please tell me if the following two products are the same one?
    Product 1:
    Title: Xerox 106R01162 25k Yellow Toner
Description: FeaturesPage yield25000 pagesBrand compatibilityXeroxCompatible productsPhaser 7760Colour of productBlackQuantity1Printing coloursYellowTypeTonerPackaging dataPackaging width95 mmPackaging height89 mmPackage weight680 gPackaging length368 mmPackaging
    Product 2:
    Title: 106R01162 Toner yellow pentru Phaser 7760
Description: yellow, 25000p for Phaser 7760
A: Let's think step by step! First, let's compare the titles of the two products. They both mention "106R01162" and "yellow toner" for the Phaser 7760 printer. So, it seems like they could be the same product.

Next, let's compare the descriptions. Both mention a page yield of 25000 pages and that the toner is yellow. The first product has more detailed information about the brand compatibility, packaging dimensions, and weight. The second product is more concise and only mentions the page yield and printer compatibility.

Based on this comparison, it seems likely that the two products are the same one. However, it's always a good idea to double-check with the seller or manufacturer to confirm.
Q: Could you please tell me if the following two products are the same one?
    Product 1:
    {p1}
    Product 2:
    {p2}
A: Let's think step by step!
"""
)

In [82]:
auto_cot_prompt_1 = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Could you please tell me if the following two products are the same one?
    Product 1:
    {p1}
    Product 2:
    {p2}
A: Let's think step by step! """
)

auto_cot_prompt_2 = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Are the following two products the same?
Product 1: 
{p1}
Product 2: 
{p2}

A: Let's think step by step! """
)



In [83]:
eval_prompt_2 = PromptTemplate(
    input_variables=["chain_of_thought_reasoning"],
    template="""Decide if this answer corresponds to two products matching or not.
    Please answer with "match" or "no match".
    Answer: {chain_of_thought_reasoning}
"""
)

In [84]:
def cut_length(string ,length = 250):
    return string[0:length]

In [85]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()


True

In [86]:
from langfuse.callback import CallbackHandler
handler_auto_cot = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [87]:
df_train = pd.read_csv("../datasets/training_data_50_50.csv")



# CoT-gpt3.5turbo-0301
# eval-gpt3.5turbo-0301
# correct-gpt3.5turbo-0301


In [131]:
def simple_auto_cot_prompt(auto_cot_prompt, start=0, end=10):
    for i in range(start, end):
        print(i)
        reasoning = ""
        llm = ChatOpenAI(model_name= "gpt-4-0613", temperature=0)

        product1 = "Title: " + cut_length(str(df_train["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(df_train["description_left"][i] or "none")) 
        product2 = "Title: " + cut_length(str(df_train["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(df_train["description_right"][i] or "none")) 
        

        cot_chain = LLMChain(
            llm=llm, prompt=auto_cot_prompt, callbacks=[handler_auto_cot]
        )
        reasoning += auto_cot_prompt.format_prompt(p1=product1, p2=product2).text
        try:
            answer = cot_chain.run(
                p1 = product1,
                p2 = product2,
                callbacks=[handler_auto_cot]
            )      
        except Exception as e:
            print(e)
            i -= 1
            continue
        reasoning += answer
        reasoning += "\n"

        # save reasoning to df_train in column "CoT-gpt3.5turbo-0301"
        df_train.loc[i, "CoT-gpt-4-0613"] = str(reasoning)

        eval_chain = LLMChain(
            llm=llm, prompt=eval_prompt_2, callbacks=[handler]
        )
        try:
            eval_output = eval_chain.run(
                chain_of_thought_reasoning = answer,
                callbacks=[handler]
            )
        except Exception as e:
            print(e)
            i -= 1
            continue
        
        is_match = False
        if  "No" in eval_output or "no" in eval_output:
            is_match = False
        elif ("Match" in eval_output) or ("match" in eval_output):
            is_match = True
        else:
            print(eval_output)
            break

        df_train.loc[i, "eval-gpt-4-0613"] = str(is_match)

        df_train.loc[i, "correct-gpt-4-0613"] = str(is_match == df_train["label"][i])






In [133]:
simple_auto_cot_prompt(auto_cot_prompt_2, start=0, end=0)

45
46
47
48
49


In [134]:
# save df_train to csv
df_train.to_csv("../datasets/training_data_50_50.csv", index=False)

In [143]:
def createAutoCotPrompt(n=2, model="gpt3.5turbo-0301"):
    fewShotData = pd.read_csv("../datasets/training_data_50_50.csv")
    fewShotData = fewShotData[fewShotData["correct-"+ model] == True]
    fewShotData = fewShotData.sample(n=n)
    template = ""
    for _, row in fewShotData.iterrows():
        template += row["CoT-"+ model] + "\n"
        template += "The two products " + ("match" if row["label"] == True else "do not match") + ".\n"
    template = "\n".join([s for s in template.splitlines() if s.strip()])
    template += """\nQ: Could you please tell me if the following two products are the same one?
Product 1:
{p1}
Product 2:
{p2}
A: Let's think step by step!
"""
    print(template)

    prompt = PromptTemplate(
        input_variables=["p1", "p2"],
        template=template
    )

    return prompt



print(createAutoCotPrompt(model="gpt-4-0613"))

Q: Are the following two products the same?
Product 1: 
Title: DVR 8 canale Hikvision TurboHD 5MP 4 canale alarma audio DS-7208HUHI-K1 H.265
Description: DVR 8 canale Hikvision TurboHD 5MP modelDS-7208HUHI-K14 canale audio,alarma,videoinregistreaza 8 camere video supraveghere HIKVISION TurboHDla rezolutie max 5 MP;compresie H.265+, H.265, H.264+, H.264
Product 2: 
Title: HIKVISION DS-7208HUHI-K1 5 MEGAPIXEL (5MP) HYBRID 8 CHANNEL DVR
Description: The HIKVISION DS-7208HUHI-K1 is a Fourth Generation TURBO 4.0 DVR. Capable of recording up to 5MP TVI Cameras at up to 12fps. This DVR use H.265+ and Duel stream video compression and uses a BNC interface for camera input. The DS-7208HUHI-K1 are Hybr
A: Let's think step by step! Both products have the same model number (DS-7208HUHI-K1) and are from the same brand (Hikvision). They both support 5MP resolution and use H.265 compression. They are both 8 channel DVRs. Therefore, yes, the two products are the same.
The two products match.
Q: Are th

In [91]:
# def simple_auto_cot_prompt(auto_cot_prompt, indexes=[], n=1):
#     if len(indexes) > 0:
#         n = len(indexes)
#     template = ""
#     for j in range(n):
#         if len(indexes) > 0:
#             i = indexes[j]
#         else:
#             i = i
#         llm = ChatOpenAI(model_name= "gpt-3.5-turbo-0301", temperature=0)

#         product1 = "Title: " + cut_length(str(df_train["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(df_train["description_left"][i] or "none")) 
#         product2 = "Title: " + cut_length(str(df_train["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(df_train["description_right"][i] or "none")) 
        

#         chain = LLMChain(
#             llm=llm, prompt=auto_cot_prompt, callbacks=[handler_auto_cot]
#         )
#         template += auto_cot_prompt.format_prompt(p1=product1, p2=product2).text
#         try:
#             answer = chain.run(
#                 p1 = product1,
#                 p2 = product2,
#                 callbacks=[handler_auto_cot]
#             )      
#         except Exception as e:
#             print(e)
#             i -= 1
#             continue
#         template += answer
#         template += "\n"
#     template += auto_cot_prompt.template
#     overall_prompt = PromptTemplate(
#         input_variables=["p1", "p2"],
#         template=template
#     )
#     # print(overall_prompt.template)
#     return overall_prompt




In [92]:
# print(simple_auto_cot_prompt(auto_cot_prompt_1, [4,6], 2).template)